In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

In [2]:
def info(img8):
    print img8.dtype
    print img8.shape
    print type(img8)
    print np.max(img8)
    print np.min(img8)

In [3]:
def DrawCluster(source, y, n):
    for i in range(0,n):
        mask = y[:,:] != i
        temp=np.copy(source)
        temp[mask]=0
        cv2.imwrite('/home/xli/Documents/3.1-Autofluorescence/Data_train/cluster_number_'+str(i)+'.jp2', temp)

In [ ]:
testshow='/home/xli/Documents/2-filter/File1out/test-matched-median.jp2'
imgshow=cv2.imread(testshow)

In [ ]:
info(imgshow)
subshow=imgshow[4000:8000,14000:19000,:]

In [ ]:
plt.imshow(img[4000:8000,14000:19000,:])
plt.show()

In [ ]:
testfile='/home/xli/testImages/PMD1463/PMD1464&1463-F33-2014.02.01-05.00.54_PMD1463_3_0099_lossless.jp2'
img=cv2.imread(testfile)

In [ ]:
#test
subimage=img[4000:8000,14000:19000,:]
data=subimage.reshape([subimage.shape[0]*subimage.shape[1],3])
y_redicted=KMeans(n_clusters = 20,max_iter=1000,tol=0.0001).fit_predict(data)

y=y_redicted.reshape([subimage.shape[0], subimage.shape[1]])

def DrawCluster(source, y, n):
    for i in range(0,n):
        mask = y[:,:] != i
        temp=np.copy(source)
        temp[mask]=0
        cv2.imwrite('/home/xli/Documents/3.1-Autofluorescence/Data_train/cluster_number_'+str(i)+'.jp2', temp)

DrawCluster(subshow,y,20)

cv2.imwrite('/home/xli/Documents/3.1-Autofluorescence/Data_train/source.jp2', subimage)

In [4]:
#generate training data:
testfile='/home/xli/testImages/PMD1463/PMD1464&1463-F33-2014.02.01-05.00.54_PMD1463_3_0099_lossless.jp2'
img=cv2.imread(testfile,-1)
subimage=img[10500:12500,10000:12000,:]
data=subimage.reshape([subimage.shape[0] * subimage.shape[1],3])



In [5]:
showfile='/home/xli/Documents/2-filter/File1out/test-matched-median.jp2'
showimg=cv2.imread(showfile)
showsub=showimg[10500:12500,10000:12000,:]

In [6]:
y_predict=KMeans(n_clusters = 3,max_iter=500,tol=0.001).fit_predict(data)
y = y_predict.reshape([subimage.shape[0], subimage.shape[1]])
#DrawCluster(showsub,y,3)

In [7]:
def ExtractF(img,x,y):
    #extract feature for pixel img[x,y]
    #check boundary
    
    res=[]
    #res.append(None)
    res.append(img[x,y,0])
    res.append(img[x,y,1])
    res.append(img[x,y,2])

    res.append(np.mean(img[x-1:x+1,y-1:y+1,0]))
    res.append(np.mean(img[x-1:x+1,y-1:y+1,1]))
    res.append(np.mean(img[x-1:x+1,y-1:y+1,2]))
    
    res.append(np.std(img[x-1:x+1,y-1:y+1,0]))
    res.append(np.std(img[x-1:x+1,y-1:y+1,1]))
    res.append(np.std(img[x-1:x+1,y-1:y+1,2]))
    
    res.append(np.mean(img[x-2:x+2,y-2:y+2,0]))
    res.append(np.mean(img[x-2:x+2,y-2:y+2,1]))
    res.append(np.mean(img[x-2:x+2,y-2:y+2,2]))
    
    res.append(np.std(img[x-2:x+2,y-2:y+2,0]))
    res.append(np.std(img[x-2:x+2,y-2:y+2,1]))
    res.append(np.std(img[x-2:x+2,y-2:y+2,2]))
    
    return res

In [8]:
print ExtractF(img,1000,1000)

[0, 14, 3, 3.75, 14.5, 9.25, 2.8613807855648994, 9.2330926563096938, 6.3393611665529832, 4.0, 10.125, 8.625, 5.123475382979799, 6.9180470510108556, 5.9358550352918833]


In [9]:
#per pixel:
dataX=[]
datay=[]
for i in range(0,subimage.shape[0]):
    for j in range(0, subimage.shape[1]):
        
        #check boundary
        if i > 0+2 and j > 0+2 and i < subimage.shape[0]-2 and j < subimage.shape[1]-2:
            dataX.append(ExtractF(subimage,i,j))
            if y[i,j] == 2:
                datay.append(1)
            else:
                datay.append(-1)

In [20]:
trainX=dataX[0:30000]
trainy=datay[0:30000]

testX=dataX[300000:400000]
testy=datay[300000:400000]

In [21]:
from sklearn import svm

In [23]:
clf = svm.SVC(verbose=True)
clf.fit(trainX,trainy)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True)

In [14]:
def diff(y_predicted, testy):
    #return tha # of different value
    count=0
    for i in range(0,len(testy)):
        if y_predicted[i] != testy[i]:
            count = count + 1
    return count

In [24]:
y_p=clf.predict(testX)

In [25]:
diff(y_p,testy)

9757

In [26]:
4702/float(100000)

0.04702